In [ ]:
import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import glob, os

import itertools

from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML

import statsmodels.tsa.api as smt
import statsmodels.api as sm

import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go

import pandas as pd
import numpy as np
import scipy
import pprint

%pylab inline
figsize(20,10)

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)


df = pd.DataFrame()

for e in exo_list:
    exo_series, exo_dict = storage.load_series(e)
    
    exo_series = exo_series[exo_series.columns[0]]
    df[exo_dict['name']] = exo_series
#exo_series.plot()

In [ ]:
df.tail()

In [ ]:
px_ser = df.ZN_PutSpread
px_ser.asfreq('B').dropna().plot()

In [ ]:
def hurst(ts):
    """Returns the Hurst Exponent of the time series vector ts"""
    # Create the range of lag values
    lags = range(2, 20)
    # Calculate the array of the variances of the lagged differences
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]
    # Use a linear fit to estimate the Hurst Exponent
    poly = polyfit(log(lags), log(tau), 1)
    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0

In [ ]:
rolling_hurst = px_ser.dropna().rolling(40).apply(lambda x: hurst(x))

In [ ]:
px_ser.dropna().ix['2015'].plot()
rolling_hurst.ix['2015'].plot(secondary_y=True)

#rolling_adfteststats.ix[:'2015'].plot(secondary_y=True)
#rstats.plot(secondary_y=True)
#axhline(rolling_hurst.mean())
#axhline(-1)

In [ ]:
px_ser.dropna().ix['2012'].plot()
#px_ser.rolling(10).max().plot()
#px_ser.rolling(10).min().plot()

rolling_hurst.ix['2012'].plot(secondary_y=True)
rolling_hurst.rolling(5).median().ix['2012'].plot(secondary_y=True)

# Backtest

In [ ]:
#entry = signals_df.long_entry_signal
#exit = signals_df.long_exit_signal

entry = (rolling_hurst <= 0.3)

exit = (rolling_hurst < rolling_hurst.rolling(20).median())
#exit = CrossUp(px_ser, px_ser.rolling(10).median())

exodf = pd.DataFrame()
exodf['exo'] = px_ser.dropna()

direction = -1
pl, inposition = backtester.backtest(exodf.iloc[:], entry, exit, direction)
equity, stats = backtester.stats(pl, inposition)


equity.plot()

(px_ser*-1).plot()

stats

In [ ]:
(px_ser - px_ser.rolling(10).min())

In [ ]:
px_ser.rolling(20).std().plot()
(rolling_hurst <= 0.).plot(secondary_y=True)